### Import Data

In [1]:
import pandas as pd

In [21]:
path = '../disaster_npl_prediction/'

In [22]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')

In [23]:
train = train[['text', 'target']]

In [24]:
train['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [25]:
train.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train['text']

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
5       #RockyFire Update => California Hwy. 20 closed...
6       #flood #disaster Heavy rain causes flash flood...
7       I'm on top of the hill and I can see a fire in...
8       There's an emergency evacuation happening now ...
9       I'm afraid that the tornado is coming to our a...
10            Three people died from the heat wave so far
11      Haha South Tampa is getting flooded hah- WAIT ...
12      #raining #flooding #Florida #TampaBay #Tampa 1...
13                #Flood in Bago Myanmar #We arrived Bago
14      Damage to school bus on 80 in multi car crash ...
15                                         What's up man?
16                                          I love fruits
17            

### Remove URL, RT, mention(@)

In [47]:
train.text = train.text.str.replace(r'http(\S)+', r'')
train.text = train.text.str.replace(r'http ...', r'')

In [48]:
train.text[train.text.str.contains(r'http')]

121    Aftershock: Protect Yourself and Profit in the...
Name: text, dtype: object

In [49]:
train.text[121]

'Aftershock: Protect Yourself and Profit in the Next Global Financial Meltdown by David Wiedemer http '

In [43]:
train.text = train.text.str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r'')

In [45]:
train.text[train.text.str.contains(r'RT[ ]?@')]

Series([], Name: text, dtype: object)

In [50]:
train.text = train.text.str.replace(r'@[\S]+', r'')

In [52]:
train.text[train.text.str.contains(r'@[\S]')]

Series([], Name: text, dtype: object)

### Remove extra space

In [55]:
train.text = train.text.str.replace(r'[ ]{2, }',r' ')

### &, < and >

In [56]:
train.text = train.text.str.replace(r'&amp;?', r'and')

In [57]:
train.text = train.text.str.replace(r'&lt;',r'<')
train.text = train.text.str.replace(r'&gt;',r'>')

### Insert space between words and punctuation marks

In [58]:
train.text = train.text.str.replace(r'([\w\d]+)([^\w\d ]+)', r'\1 \2')

In [59]:
train.text = train.text.str.replace(r'([^\w\d ]+)([\w\d]+)', r'\1 \2')

### Lowercased and strip

In [60]:
train.text = train.text.str.lower()
train.trext = train.text.str.strip()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [61]:
train.head()

,text,target
0,our deeds are the reason of this # earthquake ...,1
1,forest fire near la ronge sask . canada,1
2,all residents asked to ' shelter in place ' ar...,1
3,"13 , 000 people receive # wildfires evacuation...",1
4,just got sent this photo from ruby # alaska as...,1


In [67]:
train['ProcessedText_length'] = [len(texts.split(' ')) for texts in train.text]


In [70]:
train.head()

,text,target,ProcessedText_length
0,our deeds are the reason of this # earthquake ...,1,14
1,forest fire near la ronge sask . canada,1,8
2,all residents asked to ' shelter in place ' ar...,1,25
3,"13 , 000 people receive # wildfires evacuation...",1,12
4,just got sent this photo from ruby # alaska as...,1,19


In [72]:
train['ProcessedText_length'].value_counts()

14    398
20    392
22    387
23    385
17    365
21    341
18    341
15    334
11    328
13    326
19    326
16    323
12    323
24    312
25    310
10    261
9     252
26    242
27    197
8     188
28    173
7     163
29    143
6     143
5     122
30    114
31     92
4      82
32     53
3      52
33     37
2      23
34     21
35     20
36     12
42      6
37      5
1       5
43      5
40      3
38      3
41      2
44      1
39      1
59      1
Name: ProcessedText_length, dtype: int64

### Drop texts with length <=3 and drop duplicates

In [75]:
train = train[train['ProcessedText_length']>3]

In [76]:
train = train.drop_duplicates(subset=['text'])

In [81]:
train = train.drop(columns=['ProcessedText_length'])

In [83]:
train.head()

,text,target
0,our deeds are the reason of this # earthquake ...,1
1,forest fire near la ronge sask . canada,1
2,all residents asked to ' shelter in place ' ar...,1
3,"13 , 000 people receive # wildfires evacuation...",1
4,just got sent this photo from ruby # alaska as...,1


In [82]:
train.shape

(6871, 2)

In [84]:
train.target.value_counts()

0    4037
1    2834
Name: target, dtype: int64

### BERT preprocess

In [85]:
train['ProcessedText_BERT'] = '[CLS] '+train.text


In [87]:
!pip install pytorch-pretrained-bert

    100% |████████████████████████████████| 133kB 870kB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 1.8MB/s ta 0:00:011
    100% |████████████████████████████████| 686kB 15.3MB/s ta 0:00:01
    100% |████████████████████████████████| 5.9MB 4.9MB/s eta 0:00:01
  Stored in directory: /Users/augustinechang/Library/Caches/pip/wheels/98/16/eb/136427a471a706ea4847904c4b37c27a21bc3c90763092b67c
Successfully built regex
awscli 1.16.203 has requirement botocore==1.12.193, but you'll have botocore 1.13.46 which is incompatible.
  Found existing installation: botocore 1.12.193
    Uninstalling botocore-1.12.193:
      Successfully uninstalled botocore-1.12.193


In [88]:
from pytorch_pretrained_bert import BertTokenizer

In [89]:
train.head()

,text,target,ProcessedText_BERT
0,our deeds are the reason of this # earthquake ...,1,[CLS] our deeds are the reason of this # earth...
1,forest fire near la ronge sask . canada,1,[CLS] forest fire near la ronge sask . canada
2,all residents asked to ' shelter in place ' ar...,1,[CLS] all residents asked to ' shelter in plac...
3,"13 , 000 people receive # wildfires evacuation...",1,"[CLS] 13 , 000 people receive # wildfires evac..."
4,just got sent this photo from ruby # alaska as...,1,[CLS] just got sent this photo from ruby # ala...


In [90]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train['ProcessedText_BERTbase_length'] = [len(tokenizer.tokenize(sent)) for sent in train.ProcessedText_BERT]

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
train['ProcessedText_BERTlarge_length'] = [len(tokenizer.tokenize(sent)) for sent in train.ProcessedText_BERT]

100%|██████████| 231508/231508 [00:00<00:00, 2505120.92B/s]


### Int label for later use in softmax and cross entropy loss

In [91]:
label_dict = dict()
for i, l in enumerate(list(train.text.value_counts().keys())):
    label_dict.update({l: i})

train['InformationType_label'] = [label_dict[label] for label in train.text]

### Save data

In [92]:
train.to_csv('preprocessed_data.csv')